In [1]:
load("ckks.py")

In [2]:
N = 2**15  # Ring degree
n = 2**2  # Number of slots
L = 20  # Number of levels
q0 = 2**40  # Smallest modulus
delta = 2**33  # Scaling factor

# The grouping parameter for bootstrapping is s. It lies between 1 and
# log(n, 2). The smaller s, the faster the bootstrapping, but the more levels
# are consumed.
s = log(n, 2)

CKKS.config(N, n, L, q0, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, s)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(80.0).

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-1.+3.j,  0.+3.j, -2.-2.j,  2.-1.j]),
 array([1.-1.j, 0.+1.j, 3.-2.j, 1.+2.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[- 2147483648 + 15672725838X^4096 - 9111001500X^8192 + 3486629326X^12288 + 6442450944X^16384 + 9779081255X^20480 + 6074001000X^24576 + 8417467806X^28672 mod(2^700),
 10737418240 - 4308435740X^4096 - 4555500750X^8192 - 821806413X^12288 + 6433451618X^20480 - 13666502250X^24576 + 1984016189X^28672 mod(2^700)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20),
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-1.00000000e+00+3.j  6.41018683e-10+3.j -2.00000000e+00-2.j
  2.00000000e+00-1.j]
[9.99999999e-01-1.j 6.34724939e-10+1.j 3.00000000e+00-2.j
 1.00000000e+00+2.j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^20 (level 20 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^19 (level 19 out of 20)


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[-1.29887556e-09+2.j  1.27574373e-09+4.j  1.00000000e+00-4.j
  3.00000000e+00+1.j]
[  2.+4.00000000e+00j  -3.+1.77124115e-09j -10.-2.00000000e+00j
   4.+3.00000000e+00j]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(s)
print(ct_boot)

pt = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^0 (level 0 out of 20)
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^40) * (2^33)^2 (level 2 out of 20)
[-1.00110891e+00+2.99860749j  8.98769732e-04+2.99708707j
 -1.99782176e+00-1.99829159j  1.99806595e+00-0.99808021j]
